In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the range for X[:,0] and X[:,1]
x0_range = np.linspace(0, 100000, 100)  # X[:,0] values
x1_range = np.linspace(0, 100000, 100)  # X[:,1] values

# Fixed value for X[:,2]
x2_fixed = 50000

# Generate meshgrid for X[:,0] and X[:,1]
x0, x1 = np.meshgrid(x0_range, x1_range)

# Calculate y using the given equation with fixed X[:,2]
y_values = np.abs(
    np.exp(0.00001 * x0) * np.sin(x0 * 0.0001) + 
    np.cos(x1 * 0.0001) * np.exp(0.00001 * x1) - 
    0.00001 * x2_fixed + 
    np.random.rand(*x0.shape) * 0.5
)

# Plotting the heat map
plt.figure(figsize=(10, 8))
plt.imshow(y_values, extent=(0, 100000, 0, 100000), origin='lower', cmap='viridis', aspect='auto')
plt.colorbar(label='y values')
plt.xlabel('X[:,0] (Weight Job Wait Time)')
plt.ylabel('X[:,1] (Weight for Fair Share)')
plt.title('Heatmap of y with X[:,2] (Weight for Job Size) fixed at 50000')
plt.show()

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import matplotlib.pyplot as plt

# Generating synthetic data for the model
np.random.seed(42)
X = np.random.randint(0, 100000, (1000, 3))  # 100 samples, 3 parameters

# Modifying the response to ensure all values are positive by applying exponential transformations
#y = np.exp(0.00001 * X[:, 0]) + np.exp(0.00001 * X[:, 1]) - 0.00001 * X[:, 2] + np.random.rand(100) * 0.5
y = np.abs(np.exp(0.00001 * X[:, 0]) * np.sin(X[:, 0] * 0.0001) + np.cos(X[:, 1] * 0.0001) * np.exp(0.00001 * X[:, 1]) - 0.00001 * X[:, 2] + np.random.rand(1000) * 0.5)
#print(X)
#print(y)
# Gaussian Process Model
kernel = C(1.0, (1e-4, 1e1)) * RBF([1e4, 1e4, 1e4], (1e2, 1e6))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-10)
gp.fit(X, y)

# Creating a grid of points for prediction and visualization
grid_x, grid_y = np.meshgrid(np.linspace(0, 100000, 100), np.linspace(0, 100000, 100))
grid = np.stack([grid_x.ravel(), grid_y.ravel(), np.full(10000, 50000)]).T  # Fixing the third parameter for visualization
y_pred = gp.predict(grid).reshape(100, 100)
y_pred = np.maximum(0, y_pred)

# Plotting the 3D thermal plot
plt.figure(figsize=(10, 8))
plt.imshow(y_pred, extent=(0, 100000, 0, 100000), origin='lower', cmap='hot')
plt.colorbar(label='Predicted Wait Time')
plt.xlabel('Weight Job Wait Time')
plt.ylabel('Weight for Fair Share')
plt.title('Predicted Wait Time Surface for Fixed Job Size Weight = 50000')
plt.show()


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Generating synthetic data for the model
np.random.seed(42)
X = np.random.randint(0, 100000, (100, 3))  # 100 samples, 3 parameters
# Adjusting the response function to ensure non-negative values
y = abs(np.exp(0.00001 * X[:, 0]) * np.sin(X[:, 0] * 0.0001) + np.cos(X[:, 1] * 0.0001) * np.exp(0.00001 * X[:, 1]) - 0.00001 * X[:, 2] + np.random.rand(100) * 0.5)

#print(X)
#print(y)
# Define a kernel with parameter bounds
kernel = C(1.0, (1e-4, 1e1)) * RBF([1e4, 1e4, 1e4], (1e2, 1e6))

# Create the Gaussian Process model
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-10)
gp.fit(X, y)

# Creating a grid of points for all parameters to find the optimal settings
grid_x, grid_y, grid_z = np.meshgrid(
    np.linspace(0, 100000, 50),  # Weight Job Wait Time
    np.linspace(0, 100000, 50),  # Weight for Fair Share
    np.linspace(0, 100000, 50)   # Weight for Job Size
)
grid = np.stack([grid_x.ravel(), grid_y.ravel(), grid_z.ravel()]).T
y_pred_full = gp.predict(grid).reshape(grid_x.shape)
y_pred_full = np.maximum(0, y_pred_full)

# Finding the minimal wait time and corresponding optimal parameters
min_idx_full = np.unravel_index(np.argmin(y_pred_full), y_pred_full.shape)
optimal_params = {
    "Weight Job Wait Time": grid_x[min_idx_full],
    "Weight for Fair Share": grid_y[min_idx_full],
    "Weight for Job Size": grid_z[min_idx_full],
    "Minimal Predicted Wait Time": y_pred_full[min_idx_full]
}

# Print the optimal parameters and minimal predicted wait time
print("Optimal Parameters and Minimal Wait Time:", optimal_params)


In [ ]:

grid_x, grid_y, grid_z = np.meshgrid(np.linspace(0, 100000, 50), np.linspace(0, 100000, 50), np.linspace(0, 100000, 50))
grid = np.column_stack([grid_x.ravel(), grid_y.ravel(), grid_z.ravel()])

# Predict over the new grid
grid = np.stack([grid_x.ravel(), grid_y.ravel(), grid_z.ravel()]).T
y_pred = gp.predict(grid).reshape(grid_x.shape)
#y_pred = gp.predict(grid).reshape(100, 100, 100)

# Find the index of the maximum value in the predicted outcomes
max_index = np.unravel_index(np.argmax(y_pred), y_pred.shape)
optimal_job_wait_time = grid_x[max_index]
optimal_fair_share_weight = grid_y[max_index]
optimal_job_size_weight = grid_z[max_index]

print("Optimal Weight Job Wait Time:", optimal_job_wait_time)
print("Optimal Weight for Fair Share:", optimal_fair_share_weight)
print("Optimal Weight for Job Size:", optimal_job_size_weight)